# <font color = red> Project: 
## Predict Kidney Tumor Stage With Machine Learning Using Epigenomic Data 

# <font color = red> Problem: 
## High misdiagnosis rate of tumor stage, which is important for doctors to choose suitable therapy for patients

# <font color = red> Data Pipeline:
## 1. Download data sets from TCGA database: (i). Patients' clinical diagnosis data (ii). DNA          methylation data
## 2. Cleaning and merging two datasets
## 3. Feature Selection: (i). PCA (ii). LDA (iii). Kruskal-Wallis Method
## 4. Classifier: (i). LDA (ii). Logistic Regression (iii). Random Forest 

## <font color = red> Results:
### 1. Tested several different combinations of dimension reduction methods (PCA, LDA, Kruskal- Wallis test) and classifiers (Logistic regression, LDA, Random forest) in Python; combinations of Kruskal-Wallis test and random forest classifier have highest accuracy in predicting tumor stage 
### 2. Analyzed 208 patients with over 20,000 genes methylation data (from TCGA database) and identified 20 genes as key features for tumor stage prediction; accuracy of prediction is over 70%

## Part 1. Data Loading, Cleaning, and Merge

In [1]:
## Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1.1 Clinical Diagnosis Data

In [2]:
## First dataset: Patients' clinical diagnosis data 
data1 = pd.read_csv('KIRC.clin.merged.picked.txt', delimiter="\t")

## Get pathological stage data
clinical = data1.iloc[6:7,:]
clinical = clinical.rename(columns = {'Hybridization REF':'Patient_Barcode'}) # df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})

## Transpose data & rename
B = clinical.transpose()
B.head()

# B.columns = ["pathologic_stage"]
clinical_final = B.iloc[1:,:]
clinical_final=clinical_final.reset_index()
clinical_final.columns = ["patient_barcode","pathologic_stage"]
clinical_final.head()

,patient_barcode,pathologic_stage
0,tcga-a3-3307,stage iii
1,tcga-a3-3325,stage i
2,tcga-a3-3328,stage i
3,tcga-a3-3329,stage i
4,tcga-a3-3336,stage i


### 1.2 DNA Methylation Data

In [3]:
## Second dataset: DNA Methylation Data
url = 'KIRC.methylation__humanmethylation27__jhu_usc_edu__Level_3__within_bioassay_data_set_function__data.data.txt'
meth = pd.read_csv(url, delimiter="\t")
## Choose beta-value & gene symbol
meth = meth.transpose().reset_index()
meth = meth.iloc[1:,:]

## Rename column with gene names
header = ['patient','category']+list(meth.iloc[1,2:])

## Mark different locations in the same gene as: geneA and geneA_1
# Ex: ATP2A1 and ATP2A1_1
m = set()
for n in range(len(header)):
    if str(header[n]) in m:
        header[n] = str(header[n])+'_1'
        m.add(header[n])
    else:
        m.add(header[n])
meth = meth.set_axis(header, axis='columns', inplace=False)

## Drop rows with 'Chromosome' & 'Genomic Coordinate' & 'Gene_Symbol' item in 'category' column
meth = meth[~meth.category.str.contains('Chromosome')]
meth = meth[~meth.category.str.contains('Genomic_Coordinate')]
meth = meth[~meth.category.str.contains('Gene_Symbol')]

/home/czl/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,5,8,9,12,13,16,17,20,21,24,25,28,29,32,33,36,37,40,41,44,45,48,49,52,53,56,57,60,61,64,65,68,69,72,73,76,77,80,81,84,85,88,89,92,93,96,97,100,101,104,105,108,109,112,113,116,117,120,121,124,125,128,129,132,133,136,137,140,141,144,145,148,149,152,153,156,157,160,161,164,165,168,169,172,173,176,177,180,181,184,185,188,189,192,193,196,197,200,201,204,205,208,209,212,213,216,217,220,221,224,225,228,229,232,233,236,237,240,241,244,245,248,249,252,253,256,257,260,261,264,265,268,269,272,273,276,277,280,281,284,285,288,289,292,293,296,297,300,301,304,305,308,309,312,313,316,317,320,321,324,325,328,329,332,333,336,337,340,341,344,345,348,349,352,353,356,357,360,361,364,365,368,369,372,373,376,377,380,381,384,385,388,389,392,393,396,397,400,401,404,405,408,409,412,413,416,417,420,421,424,425,428,429,432,433,436,437,440,441,444,445,448,449,452,453,456,457,460,461,464,465,468,469,472,

### 1.3 Merge Datasets

In [4]:
## Merge two datasets based on patient ID
    # 1. Create a new column named column, which contains first 12 letters of patient_code
meth['col'] = meth['patient'].str[0:12]
meth['col_lower'] = meth['col'].str.lower() # Convert 'col' column to lower case
meth_final = meth.rename(columns = {'col_lower':'patient_barcode'})

# patient: TCGA-A3-3306-11A (tumor sample) v.s. TCGA-A3-3306-01A (normal sample)
# https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/

    # 2.Merge clinical data & DNA methylation data
mixed_df = pd.merge(clinical_final, meth_final, how='inner', on=['patient_barcode']) # Choose overlapped region
mixed_df = mixed_df.drop(['col'],axis=1)
mixed_df.head()

,patient_barcode,pathologic_stage,patient,category,ATP2A1,SLMAP,MEOX2,HOXD3,ZNF425;ZNF398,PANX1,...,A2ML1_1,ZFP2_1,CST9L_1,C11orf24_1,nan,LEPRE1;C1orf50_1,GNAS_1,RPN1_1,CYB5A_1,AP1S1
0,tcga-a3-3325,stage i,TCGA-A3-3325-11A-01D-0859-05,Beta_value,0.341092896642809,0.163420373331899,0.0289863381040888,0.704490559626231,NaN,0.0393644321967782,...,0.800048,0.0221649,NaN,0.0195139,0.18193,0.0284293,0.389761,0.0101311,0.0167906,0.0393153
1,tcga-a3-3325,stage i,TCGA-A3-3325-01A-01D-0859-05,Beta_value,0.7253328141359,0.235892179963932,0.237244972745325,0.813347696270696,NaN,0.0513241779156893,...,0.530865,0.0235912,NaN,0.0211279,0.186888,0.0288994,0.371611,0.0100788,0.0142257,0.0132333
2,tcga-a3-3328,stage i,TCGA-A3-3328-11A-01D-0859-05,Beta_value,0.38595358991842,0.172516336695289,0.0267564273359283,0.737048613855687,NaN,0.032547823619346,...,0.800673,0.017531,NaN,0.0177181,0.185258,0.0282834,0.363502,0.0098538,0.0156143,0.0360442
3,tcga-a3-3328,stage i,TCGA-A3-3328-01A-01D-0859-05,Beta_value,0.88549631227471,0.0384832173567634,0.0235068147586301,0.109113302762268,NaN,0.0486053663743769,...,0.903917,0.017625,NaN,0.0189847,0.179154,0.05492,0.377929,0.00975378,0.0150563,0.0146956
4,tcga-a3-3329,stage i,TCGA-A3-3329-11A-01D-0859-05,Beta_value,0.257540543081615,0.126193684266256,0.0453059118170011,0.658352096657674,NaN,0.0417558123692779,...,0.786991,0.0213796,NaN,0.0199632,0.160584,0.046203,0.337231,0.0105036,0.017995,0.0418536


In [5]:
# remove Nan
mixed_final = mixed_df.dropna(axis='columns',how='any') # remove column with any Nan value!
## Detect whether there's Nan in dataframe
mixed_final.isnull().values.any() # Detect whether there's missing value in dataframe
mixed_final.isnull().sum().sum() # This returns an integer of the total number of NaN values

## Need to add "pathologic_stage" back to the dataframe (it seemed to be removed during deleting Nan)!!
mixed_final[['pathologic_stage']] = mixed_df[['pathologic_stage']] 
mixed_final.isnull().sum().sum()
mixed_final = mixed_final.dropna(axis='index',how='any')
mixed_final.isnull().sum().sum() 
mixed_final.head()

/home/czl/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,patient_barcode,patient,category,ATP2A1,SLMAP,MEOX2,HOXD3,PANX1,COX8C;KIAA1409,TTC8,...,MYCL1_1,KIAA1409;COX8C,ALDH1A3_1,ZFP2_1,C11orf24_1,LEPRE1;C1orf50_1,GNAS_1,RPN1_1,CYB5A_1,pathologic_stage
0,tcga-a3-3325,TCGA-A3-3325-11A-01D-0859-05,Beta_value,0.341092896642809,0.163420373331899,0.0289863381040888,0.704490559626231,0.0393644321967782,0.988790175005561,0.0101987967341247,...,0.034498,0.982875,0.280882,0.0221649,0.0195139,0.0284293,0.389761,0.0101311,0.0167906,stage i
1,tcga-a3-3325,TCGA-A3-3325-01A-01D-0859-05,Beta_value,0.7253328141359,0.235892179963932,0.237244972745325,0.813347696270696,0.0513241779156893,0.988732478437813,0.00849065030592262,...,0.0397872,0.97624,0.471156,0.0235912,0.0211279,0.0288994,0.371611,0.0100788,0.0142257,stage i
2,tcga-a3-3328,TCGA-A3-3328-11A-01D-0859-05,Beta_value,0.38595358991842,0.172516336695289,0.0267564273359283,0.737048613855687,0.032547823619346,0.989472879556795,0.00756174932895548,...,0.0308388,0.986335,0.360738,0.017531,0.0177181,0.0282834,0.363502,0.0098538,0.0156143,stage i
3,tcga-a3-3328,TCGA-A3-3328-01A-01D-0859-05,Beta_value,0.88549631227471,0.0384832173567634,0.0235068147586301,0.109113302762268,0.0486053663743769,0.986829026212807,0.0137776185674072,...,0.0170604,0.989409,0.700757,0.017625,0.0189847,0.05492,0.377929,0.00975378,0.0150563,stage i
4,tcga-a3-3329,TCGA-A3-3329-11A-01D-0859-05,Beta_value,0.257540543081615,0.126193684266256,0.0453059118170011,0.658352096657674,0.0417558123692779,0.987499601652852,0.0104639793481409,...,0.0259481,0.991082,0.236762,0.0213796,0.0199632,0.046203,0.337231,0.0105036,0.017995,stage i


### 1.4 Separate Tumor and Normal Samples; Label Early Stage as 1 and Label Late Stage as 2

In [6]:
## Separate tumor tissue with normal tissue
tumor_list = []
for n in range(int(mixed_final.shape[0]/2)):
    m = 2*n
    tumor_list.append(m)

tumor_df = mixed_final.iloc[tumor_list,:]

## Label tumor stage 1,2 to be group 1; tumor stage 3,4  to be group 2
dic = {'stage i':1, 'stage ii':1, 'stage iii':2, 'stage iv':2}
tumor_df['group']=tumor_df['pathologic_stage'].map(dic)
tumor_df.head()

/home/czl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,patient_barcode,patient,category,ATP2A1,SLMAP,MEOX2,HOXD3,PANX1,COX8C;KIAA1409,TTC8,...,KIAA1409;COX8C,ALDH1A3_1,ZFP2_1,C11orf24_1,LEPRE1;C1orf50_1,GNAS_1,RPN1_1,CYB5A_1,pathologic_stage,group
0,tcga-a3-3325,TCGA-A3-3325-11A-01D-0859-05,Beta_value,0.341092896642809,0.163420373331899,0.0289863381040888,0.704490559626231,0.0393644321967782,0.988790175005561,0.0101987967341247,...,0.982875,0.280882,0.0221649,0.0195139,0.0284293,0.389761,0.0101311,0.0167906,stage i,1
2,tcga-a3-3328,TCGA-A3-3328-11A-01D-0859-05,Beta_value,0.38595358991842,0.172516336695289,0.0267564273359283,0.737048613855687,0.032547823619346,0.989472879556795,0.00756174932895548,...,0.986335,0.360738,0.017531,0.0177181,0.0282834,0.363502,0.0098538,0.0156143,stage i,1
4,tcga-a3-3329,TCGA-A3-3329-11A-01D-0859-05,Beta_value,0.257540543081615,0.126193684266256,0.0453059118170011,0.658352096657674,0.0417558123692779,0.987499601652852,0.0104639793481409,...,0.991082,0.236762,0.0213796,0.0199632,0.046203,0.337231,0.0105036,0.017995,stage i,1
6,tcga-a3-3336,TCGA-A3-3336-11A-01D-0859-05,Beta_value,0.401042479758341,0.154713484311169,0.0324099413145908,0.696493921392044,0.0561693907578759,0.9894664462667,0.00809906530171172,...,0.991071,0.321022,0.0191658,0.0229501,0.0315108,0.354126,0.0112805,0.0179185,stage i,1
8,tcga-a3-3343,TCGA-A3-3343-11A-01D-0859-05,Beta_value,0.0261269841007268,0.195132806260209,0.0442763309722378,0.271151912048597,0.0895605935074786,0.985994929771935,0.0111629863693614,...,0.991888,0.227373,0.0361556,0.0309748,0.0450911,0.21732,0.00884356,0.0268121,stage ii,1


### 1.5 Export Clean Dataset

In [10]:
## Only keep genes methylation and label columns
## Export clean dataset
X_tumor = tumor_df.drop(['patient_barcode', 'pathologic_stage', 'patient', 'category'],axis=1)
X_tumor.to_csv('Tumor_stage.csv')
X_tumor.head()

,ATP2A1,SLMAP,MEOX2,HOXD3,PANX1,COX8C;KIAA1409,TTC8,nan,TMEM186;PMM2,ANG;RNASE4,...,MYCL1_1,KIAA1409;COX8C,ALDH1A3_1,ZFP2_1,C11orf24_1,LEPRE1;C1orf50_1,GNAS_1,RPN1_1,CYB5A_1,group
0,0.341092896642809,0.163420373331899,0.0289863381040888,0.704490559626231,0.0393644321967782,0.988790175005561,0.0101987967341247,0.553604734439809,0.953507442534218,0.0145064421302908,...,0.034498,0.982875,0.280882,0.0221649,0.0195139,0.0284293,0.389761,0.0101311,0.0167906,1
2,0.38595358991842,0.172516336695289,0.0267564273359283,0.737048613855687,0.032547823619346,0.989472879556795,0.00756174932895548,0.620752883476738,0.953492118622301,0.0104582276294671,...,0.0308388,0.986335,0.360738,0.017531,0.0177181,0.0282834,0.363502,0.0098538,0.0156143,1
4,0.257540543081615,0.126193684266256,0.0453059118170011,0.658352096657674,0.0417558123692779,0.987499601652852,0.0104639793481409,0.652652809982813,0.914208531588916,0.0238009339117171,...,0.0259481,0.991082,0.236762,0.0213796,0.0199632,0.046203,0.337231,0.0105036,0.017995,1
6,0.401042479758341,0.154713484311169,0.0324099413145908,0.696493921392044,0.0561693907578759,0.9894664462667,0.00809906530171172,0.615327033621039,0.949000471552509,0.0126582340654666,...,0.0427957,0.991071,0.321022,0.0191658,0.0229501,0.0315108,0.354126,0.0112805,0.0179185,1
8,0.0261269841007268,0.195132806260209,0.0442763309722378,0.271151912048597,0.0895605935074786,0.985994929771935,0.0111629863693614,0.509603947268456,0.926462406994679,0.0213652841338812,...,0.0533251,0.991888,0.227373,0.0361556,0.0309748,0.0450911,0.21732,0.00884356,0.0268121,1


## Part 2. Feature Selection & Extraction

In [7]:
## Data Loading 1 (Total data)
## From above clean dataset
X_tum = pd.read_csv('Tumor_stage.csv')
X_tumor = X_tum.drop(['Unnamed: 0'], axis=1)
y_tumor = X_tum[['group']]
X_tumor.head()

,ATP2A1,SLMAP,MEOX2,HOXD3,PANX1,COX8C;KIAA1409,TTC8,Unnamed: 8,TMEM186;PMM2,ANG;RNASE4,...,MYCL1_1,KIAA1409;COX8C,ALDH1A3_1.4,ZFP2_1,C11orf24_1,LEPRE1;C1orf50_1.1,GNAS_1.15,RPN1_1,CYB5A_1,group
0,0.341093,0.163420,0.028986,0.704491,0.039364,0.988790,0.010199,0.553605,0.953507,0.014506,...,0.034498,0.982875,0.280882,0.022165,0.019514,0.028429,0.389761,0.010131,0.016791,1
1,0.385954,0.172516,0.026756,0.737049,0.032548,0.989473,0.007562,0.620753,0.953492,0.010458,...,0.030839,0.986335,0.360738,0.017531,0.017718,0.028283,0.363502,0.009854,0.015614,1
2,0.257541,0.126194,0.045306,0.658352,0.041756,0.987500,0.010464,0.652653,0.914209,0.023801,...,0.025948,0.991082,0.236762,0.021380,0.019963,0.046203,0.337231,0.010504,0.017995,1
3,0.401042,0.154713,0.032410,0.696494,0.056169,0.989466,0.008099,0.615327,0.949000,0.012658,...,0.042796,0.991071,0.321022,0.019166,0.022950,0.031511,0.354126,0.011281,0.017918,1
4,0.026127,0.195133,0.044276,0.271152,0.089561,0.985995,0.011163,0.509604,0.926462,0.021365,...,0.053325,0.991888,0.227373,0.036156,0.030975,0.045091,0.217320,0.008844,0.026812,1


### 2.1 Train-Test Split

In [8]:
from sklearn.model_selection import train_test_split # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train_di, X_test_di, y_train_di, y_test_di = train_test_split (X_tumor, y_tumor, test_size = 0.3, stratify=X_tumor['group'])
X_train_dif = X_train_di.drop(['group'],axis=1)
X_test_dif = X_test_di.drop(['group'],axis=1)

X_train_dif.head()

,ATP2A1,SLMAP,MEOX2,HOXD3,PANX1,COX8C;KIAA1409,TTC8,Unnamed: 8,TMEM186;PMM2,ANG;RNASE4,...,DAB2IP_1.6,MYCL1_1,KIAA1409;COX8C,ALDH1A3_1.4,ZFP2_1,C11orf24_1,LEPRE1;C1orf50_1.1,GNAS_1.15,RPN1_1,CYB5A_1
25,0.527216,0.388513,0.041617,0.821631,0.082274,0.990079,0.009101,0.595033,0.948046,0.012949,...,0.014801,0.261126,0.967535,0.603313,0.017250,0.025930,0.040611,0.355288,0.012419,0.029987
124,0.423040,0.148296,0.029480,0.780460,0.076754,0.990518,0.009348,0.633822,0.910637,0.014222,...,0.246410,0.059515,0.977434,0.192991,0.016769,0.027705,0.052330,0.303668,0.009113,0.024134
12,0.391904,0.354476,0.053835,0.794001,0.056835,0.991049,0.010656,0.576181,0.918385,0.013271,...,0.007594,0.184651,0.969820,0.445231,0.032726,0.022425,0.049316,0.338151,0.009884,0.031497
185,0.544094,0.291288,0.068571,0.758822,0.046573,0.990030,0.010247,0.704257,0.928793,0.015198,...,0.147100,0.027463,0.988602,0.287220,0.032016,0.030268,0.070115,0.402414,0.011178,0.032123
140,0.382834,0.281154,0.054028,0.776502,0.051761,0.990223,0.010292,0.718880,0.933810,0.013489,...,0.138122,0.050770,0.992555,0.241898,0.038682,0.034609,0.049532,0.450248,0.013350,0.021261


### 2.2 Feature Selection: Kruskal-Wallis Method

In [9]:
from scipy import stats
## Split Early and Late Stage Samples; Only use the train test sample!
tumor_G1 = X_tumor[X_tumor.group == 1].drop(['group'],axis=1)
tumor_G2 = X_tumor[X_tumor.group == 2].drop(['group'],axis=1)
Gene = tumor_G1.columns.tolist() # Have a list of genes

## For multiple pair-comparison, type I error keep accumulating! So we need higher threshold
## Setting Different threshold for p < alpha(type I error): alpha = 5e-4, 5e-5, 1e-5
alpha = [5e-4, 5e-5, 1e-5, 5e-2]
gene_kw = [[],[],[],[]]
for n in range(len(alpha)):   
    for z in range(len(Gene)):
        G1 = tumor_G1.iloc[:,z]
        G2 = tumor_G2.iloc[:,z]
        p = stats.kruskal(G1, G2)[1]
        
        if p < alpha[n]:
            gene_kw[n].append(Gene[z])
    print(len(gene_kw[n]))    

95
20
6
3659


### 2.3 Feature Selection: LDA (Linear Discriminant Analysis)

In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
     # https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html
# Linear Discriminant Analysis (LDA):
  # A classifier with a linear decision boundary, generated by fitting class conditional densities to the data and using Bayes’ rule
  # The model fits a Gaussian density to each class, assuming that all classes share the same covariance matrix.  

lda = LDA(solver="svd", store_covariance=True) #Singular value decomposition (default). Does not compute the covariance matrix, therefore this solver is recommended for data with a large number of features.
Tumor = X_tumor.drop(['group'], axis=1)
X = Tumor[gene_kw[3]] #  whole data is too large for LDA!
y = y_tumor

lda.fit(X,y)
coef = lda.coef_

/home/czl/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/czl/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [11]:
## 1. Sort the coefficients, then select top 95, 20, and 6 features
## 2. Save index, then select corresponding genes!
g_coef = coef[0]
number = [95,20,6]
gene_lda = [[],[],[]]

for n in range(len(number)):
    index = np.argsort(g_coef)[:number[n]] # numpy.argsort(list): sort the list and output the original index
    gene_lda[n] = [Gene[i] for i in index]
    print(len(gene_lda[n]))

95
20
6
